# NanoCount command line usage

### Activate virtual environment

In [1]:
conda activate nanocount

(nanocount) 

: 1

### Running NanoCount

In [4]:
NanoCount --help

usage: NanoCount [-h] [--version] -i ALIGNMENT_FILE [-o COUNT_FILE]
                 [-b FILTER_BAM_OUT] [-l MIN_READ_LENGTH]
                 [-f MIN_QUERY_FRACTION_ALIGNED] [-t EQUIVALENT_THRESHOLD]
                 [-s SCORING_VALUE] [-c CONVERGENCE_TARGET] [-e MAX_EM_ROUNDS]
                 [-x] [-p PRIMARY_SCORE] [-a] [-3 MAX_DIST_3_PRIME]
                 [-5 MAX_DIST_5_PRIME] [-v] [-q]

NanoCount estimates transcripts abundance from Oxford Nanopore *direct-RNA
sequencing* datasets, using an expectation-maximization approach like RSEM,
Kallisto, salmon, etc to handle the uncertainty of multi-mapping reads

optional arguments:
  -h, --help            show this help message and exit
  --version             show program's version number and exit

Input/Output options:
  -i ALIGNMENT_FILE, --alignment_file ALIGNMENT_FILE
                        BAM or SAM file containing aligned ONT dRNA-Seq reads
                        including secondary and supplementary alignment
              

: 1

#### Basic command

In [5]:
NanoCount -i ./data/aligned_reads_sorted.bam -o ./output/tx_counts.tsv -3 50
head ./output/tx_counts.tsv

## Checking options and input files ##
## Initialise Nanocount ##
	Parse Bam file and filter low quality alignments
	Summary of alignments parsed in input bam file
		Valid alignments: 150,779
		Discarded unmapped alignments: 9,545
		Discarded alignment with invalid 3 prime end: 6,205
		Discarded negative strand alignments: 4,515
	Summary of reads filtered
		Reads with valid best alignment: 85,200
		Invalid secondary alignments: 58,464
		Valid secondary alignments: 4,330
		Reads with low query fraction aligned: 1,544
		Reads too short: 817
	Generate initial read/transcript compatibility index
## Start EM abundance estimate ##
	Progress: 3.00 rounds [00:00, 7.90 rounds/s]
	Exit EM loop after 3 rounds
	Convergence value: 0.0019414029372732597
## Summarize data ##
	Convert results to dataframe
	Compute estimated counts and TPM
	Write file
(nanocount) transcript_name	raw	est_count	tpm
YHR174W_mRNA	0.5930218360040271	50525.46042754311	593021.8360040272
YGR192C_mRNA	0.020697465983375175	1763.

: 1

#### Adding extra transcripts information

The `extra_tx_info` option adds a columns with the transcript lengths and also includes all the zero-coverage transcripts in the results

In [7]:
NanoCount -i ./data/aligned_reads_sorted.bam -o ./output/tx_counts.tsv --extra_tx_info
head ./output/tx_counts.tsv

## Checking options and input files ##
## Initialise Nanocount ##
	Parse Bam file and filter low quality alignments
	Summary of alignments parsed in input bam file
		Valid alignments: 153,201
		Discarded unmapped alignments: 9,545
		Discarded negative strand alignments: 4,515
		Discarded alignment with invalid 3 prime end: 3,783
	Summary of reads filtered
		Reads with valid best alignment: 85,899
		Invalid secondary alignments: 60,112
		Valid secondary alignments: 4,375
		Reads with low query fraction aligned: 1,580
		Reads too short: 798
	Generate initial read/transcript compatibility index
## Start EM abundance estimate ##
	Progress: 3.00 rounds [00:00, 7.64 rounds/s]
	Exit EM loop after 3 rounds
	Convergence value: 0.0019319339005414125
## Summarize data ##
	Convert results to dataframe
	Compute estimated counts and TPM
	Write file
(nanocount) transcript_name	raw	est_count	tpm	transcript_length
YHR174W_mRNA	0.5932131975393657	50956.42045543398	593213.1975393657	1314
YGR192C_mRNA	0.0

: 1

#### Write selected alignment to BAM file

In [8]:
NanoCount -i ./data/aligned_reads_sorted.bam -o ./output/tx_counts.tsv -b ./output/aligned_reads_selected.bam --extra_tx_info
head ./output/tx_counts.tsv

## Checking options and input files ##
## Initialise Nanocount ##
	Parse Bam file and filter low quality alignments
	Summary of alignments parsed in input bam file
		Valid alignments: 153,201
		Discarded unmapped alignments: 9,545
		Discarded negative strand alignments: 4,515
		Discarded alignment with invalid 3 prime end: 3,783
	Summary of reads filtered
		Reads with valid best alignment: 85,899
		Invalid secondary alignments: 60,112
		Valid secondary alignments: 4,375
		Reads with low query fraction aligned: 1,580
		Reads too short: 798
	Write selected alignments to BAM file
	Summary of alignments written to bam
		Alignments to select: 90,274
		Alignments written: 90,274
		Alignments skipped: 80,770
	Generate initial read/transcript compatibility index
## Start EM abundance estimate ##
	Progress: 3.00 rounds [00:00, 8.00 rounds/s]
	Exit EM loop after 3 rounds
	Convergence value: 0.0019319339005414125
## Summarize data ##
	Convert results to dataframe
	Compute estimated counts and TPM

: 1

#### Relaxing the equivalence threshold

The default value is 0.9 (90% of the alignment score of the primary alignment) but this value could be lowered to allow more secondary alignments to be included in the uncertainty calculation.
Lowering the value bellow 0.75 might not be relevant and will considerably increase the computation time.

In [11]:
NanoCount -i ./data/aligned_reads_sorted.bam -o ./output/tx_counts.tsv --equivalent_threshold 0.8
head ./output/tx_counts.tsv

## Checking options and input files ##
## Initialise Nanocount ##
	Parse Bam file and filter low quality alignments
	Summary of alignments parsed in input bam file
		Valid alignments: 153,201
		Discarded unmapped alignments: 9,545
		Discarded negative strand alignments: 4,515
		Discarded alignment with invalid 3 prime end: 3,783
	Summary of reads filtered
		Reads with valid best alignment: 85,899
		Valid secondary alignments: 50,019
		Invalid secondary alignments: 14,468
		Reads with low query fraction aligned: 1,580
		Reads too short: 798
	Generate initial read/transcript compatibility index
## Start EM abundance estimate ##
	Progress: 18.0 rounds [00:03, 4.75 rounds/s]
	Exit EM loop after 18 rounds
	Convergence value: 0.00462674973518923
## Summarize data ##
	Convert results to dataframe
	Compute estimated counts and TPM
	Write file
(NanoCount) transcript_name	raw	est_count	tpm
YHR174W_mRNA	0.5828216847430426	50063.79989774262	582821.6847430427
YGR192C_mRNA	0.015176437715561169	1303.

: 1

#### verbose mode

Print additional information for QC and debugging

In [9]:
NanoCount -i ./data/aligned_reads_sorted.bam -o ./output/tx_counts.tsv --equivalent_threshold 0.8  --verbose

## Checking options and input files ##
	[DEBUG]: Options summary
	[DEBUG]: 	Package name: NanoCount
	[DEBUG]: 	Package version: 0.2.4.post1
	[DEBUG]: 	Timestamp: 2021-06-27 15:29:53.995009
	[DEBUG]: 	alignment_file: ./data/aligned_reads_sorted.bam
	[DEBUG]: 	count_file: ./output/tx_counts.tsv
	[DEBUG]: 	filter_bam_out: 
	[DEBUG]: 	min_read_length: 50
	[DEBUG]: 	discard_suplementary: False
	[DEBUG]: 	min_query_fraction_aligned: 0.5
	[DEBUG]: 	equivalent_threshold: 0.8
	[DEBUG]: 	scoring_value: alignment_score
	[DEBUG]: 	convergence_target: 0.005
	[DEBUG]: 	max_em_rounds: 100
	[DEBUG]: 	extra_tx_info: False
	[DEBUG]: 	primary_score: primary
	[DEBUG]: 	max_dist_3_prime: 100
	[DEBUG]: 	max_dist_5_prime: -1
	[DEBUG]: 	verbose: True
	[DEBUG]: 	quiet: False
## Initialise Nanocount ##
	Parse Bam file and filter low quality alignments
	Summary of alignments parsed in input bam file
		Valid alignments: 153,201
		Discarded unmapped alignments: 9,545
		Discarded negative strand alignments: 4,515
	

: 1